## 5. Python 对象的内部工作原理
本节介绍 Python 对象的内部工作原理。来自其它语言的程序员通常会发现 Python 的类概念缺乏特性。例如，没有访问控制（access-control）的概念（如：private，protected），self 参数让人感觉很奇怪，并且，坦白地说，使用对象有时候让人感到“一切都是开放的”。虽然某种程度上来说也许是这样，但是我们要了解其工作原理，以及一些常见的编程习惯，以便更好的封装对象的内部。

虽然没有必要担心内部细节会影响效率，但是，大多数 Python 程序员对类的工作原理都有基本了解。这就是为什么要介绍 Python 对象的内部工作原理的原因。
### 01 再谈字典
Python 对象系统主要基于字典实现。本节将对此进行讨论。
#### **字典**
字典是命名值（named values）的集合。
```
stock = {
    'name' : 'GOOG',
    'shares' : 100,
    'price' : 490.1
}
```
虽然字典常用于简单的数据结构，但是字典也用于解释器的关键部分。字典可能是 Python 中最重要的数据类型。
#### **字典和模块**
在模块内，字典存储所有的全局变量和函数。
```
# foo.py

x = 42
def bar():
    ...

def spam():
    ...
```
可以通过 foo.__dict__ 或 globals() 查看该字典。
```
{
    'x' : 42,
    'bar' : <function bar>,
    'spam' : <function spam>
}
```
#### **字典和对象**
用户定义对象的时候也使用到了实例字典和类字典。事实上，整个对象系统主要是基于字典实现的。

字典存储实例数据，如 __dict__：
```
>>> s = Stock('GOOG', 100, 490.1)
>>> s.__dict__
{'name' : 'GOOG', 'shares' : 100, 'price': 490.1 }
```
当给 self 赋值的时候，你将填充该字典（和实例）。
```
class Stock:
    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price
```
实例数据 self.__dict__ 看起来像下面这样：
```
{
    'name': 'GOOG',
    'shares': 100,
    'price': 490.1
}
```
每一个实例都拥有自己的私有字典。
```
s = Stock('GOOG', 100, 490.1)     # {'name' : 'GOOG','shares' : 100, 'price': 490.1 }
t = Stock('AAPL', 50, 123.45)     # {'name' : 'AAPL','shares' : 50, 'price': 123.45 }
```
如果你创建了某个类的 100 个实例，那么就会有 100 个存储数据的字典。

#### **类成员**
一个单独的字典也存储方法：
```
class Stock:
    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price

    def cost(self):
        return self.shares * self.price

    def sell(self, nshares):
        self.shares -= nshares
```
使用 `Stock.__dict__` 可以查看该字典：
```
{
    'cost': <function>,
    'sell': <function>,
    '__init__': <function>
}
```
#### **实例和类**
实例和类是链接在一起的。实例通过 `__class__` 属性指向类。
```
>>> s = Stock('GOOG', 100, 490.1)
>>> s.__dict__
{ 'name': 'GOOG', 'shares': 100, 'price': 490.1 }
>>> s.__class__
<class '__main__.Stock'>
>>>
```
实例字典存储的数据对每个实例而言是唯一的。但是，类字典存储的数据被该类的所有实例共享。
#### **属性访问**
使用对象时，可以通过 . 运算符访问数据和方法。
```
x = obj.name          # Getting
obj.name = value      # Setting
del obj.name          # Deleting
```
这些操作直接与字典绑定到一起。
#### **修改实例**
修改对象的操作会更新底层字典：
```
>>> s = Stock('GOOG', 100, 490.1)
>>> s.__dict__
{ 'name':'GOOG', 'shares': 100, 'price': 490.1 }
>>> s.shares = 50       # Setting
>>> s.date = '6/7/2007' # Setting
>>> s.__dict__
{ 'name': 'GOOG', 'shares': 50, 'price': 490.1, 'date': '6/7/2007' }
>>> del s.shares        # Deleting
>>> s.__dict__
{ 'name': 'GOOG', 'price': 490.1, 'date': '6/7/2007' }
>>>
```
#### **读取属性**
假设你要读取实例上的属性：
```
x = obj.name
```
该属性可能位于两个地方：
* 局部实例字典
* 类字典
两种字典都会被检查到。首先，检查局部实例字典 `__dict__`。如果没有找到，通过 `__class__` 查找类字典 `__dict__`。
```
>>> s = Stock(...)
>>> s.name
'GOOG'
>>> s.cost()
49010.0
>>>
```
通过这样的查找模式，类成员被所有实例共享。

#### **继承的工作原理**
一个类可能继承自其它类：
```
class A(B, C):
    ...
```
在每个类中，父类存储在一个元组中：
```
>>> A.__bases__
(<class '__main__.B'>, <class '__main__.C'>)
>>>
```
子类通过 `__bases__` 属性可以链接到父类。
#### **多继承中的属性查找**
从逻辑上讲，查找属性的过程如下：首先，检查局部字典 `__dict__`。如果没有找到，检查类字典 `__dict__`。如果在类中还是没有找到，通过 `__bases__` 属性在父类中查找。这里面有一些小细节，我们接下来讨论。
#### **单继承中的属性查找**
在继承层级结构中，通过按顺序遍历继承树来找到属性。
```
class A: pass
class B(A): pass
class C(A): pass
class D(B): pass
class E(D): pass
```
在单继承中，因为到达上层父类的路径只有一条，所以当找到第一个匹配的属性时即可停止。
#### **方法解析顺序（MRO）**
Python 会预先计算继承链并将其存储到类的 MRO 属性中。你可以像这样查看：
```
>>> E.__mro__
(<class '__main__.E'>, <class '__main__.D'>,
 <class '__main__.B'>, <class '__main__.A'>,
 <type 'object'>)
>>>
```
该继承链称为**方法解析顺序（Method Resolution Order）**。为了找到属性，Python 按顺序遍历 MRO，第一个匹配的属性即是要找的属性。（译注：有关 MRO 的更多信息，请查看 https://www.python.org/download/releases/2.3/mro/）。
多继承中的方法解析顺序

使用多继承时，到达上层父类的路径有很多条，请看示例：
```
class A: pass
class B: pass
class C(A, B): pass
class D(B): pass
class E(C, D): pass
```
访问属性时会发生什么？
```
e = E()
e.attr
```
会执行属性查找，那么按什么顺序查找呢？这是个问题。

Python 使用的是 协作多重继承（cooperative multiple inheritance），协作多继承遵守的类排序规则如下：
* 总是在检查父类之前检查子类
* 父类（如果有多个）总是按照列出的顺序检查
根据该规则， 通过按层级结构对所有的类进行排序，然后计算出方法解析顺序。
```
>>> E.__mro__
(
  <class 'E'>,
  <class 'C'>,
  <class 'A'>,
  <class 'D'>,
  <class 'B'>,
  <class 'object'>)
>>>
```
底层算法称为“C3线性化算法（C3 Linearization Algorithm）”，确切的细节不重要，只要记住类层级结构遵守的排序规则与你家房子着火后必须撤离时遵守的规则相同：首先是孩子，其次是父母。
#### **奇怪的代码重用（涉及多继承）**
考虑以下两个完全不相关的对象：
```
class Dog:
    def noise(self):
        return 'Bark'

    def chase(self):
        return 'Chasing!'

class LoudDog(Dog):
    def noise(self):
        # Code commonality with LoudBike (below)
        return super().noise().upper()
```
和
```
class Bike:
    def noise(self):
        return 'On Your Left'

    def pedal(self):
        return 'Pedaling!'

class LoudBike(Bike):
    def noise(self):
        # Code commonality with LoudDog (above)
        return super().noise().upper()
```
`LoudDog.noise()` 方法和`LoudBike.noise()` 方法中有一些通用的代码。事实上，这些通用的代码是完全一样的。自然，这样的代码势必会吸引软件工程师。
#### **"Mixin" 模式**
Mixin 模式（pattern）是包含一部分代码片段的类。
```
class Loud:
    def noise(self):
        return super().noise().upper()
```
该类不能单独使用。通过继承和其它类混合使用。
```
class LoudDog(Loud, Dog):
    pass

class LoudBike(Loud, Bike):
    pass
```
神奇的是，`noise()` 方法只实现了一次，却在两个完全不相关的类中使用。这种技巧是 Python 多继承的主要用途之一。
#### **为什么使用 super()**
当要覆盖一个方法的时候，总是使用 super() 函数。
```
class Loud:
    def noise(self):
        return super().noise().upper()
```
`super()` 函数代表 `MRO` 中的下一个类（译注：LoudDog 的 MRO 是 `LoudDog>Loud>Dog>object`。因为 Loud 的父类 object 没有定义 noise() 方法，所以 LoudDog 的实例在 Loud 中找不到 noise() 方法。然后 LoudDog 的实例就会到 MRO 中 Loud 的下一个类 Dog 中寻找）。

麻烦的是你不知道它是什么，尤其是使用多继承的时候。
#### **注意事项**
多继承是一种强大的机制。使用这种强大的机制时请牢记“权利越大，责任越大”。有时候，框架或者库使用多继承来实现一些高级特性，如组件组合。

### 练习
在第 4 节中，定义了一个表示股票持有信息的类 Stock。在本节练习中，我们将使用该类。请重新启动解释器并创建一些 Stock 类的实例：
```
>>> ================================ RESTART ================================
>>> from stock import Stock
>>> goog = Stock('GOOG',100,490.10)
>>> ibm  = Stock('IBM',50, 91.23)
>>>
```
#### **练习 5.1：实例的表示**
在交互式 shell 中，检查 goog 和 ibm 两个实例的底层字典：
```
>>> goog.__dict__
... look at the output ...
>>> ibm.__dict__
... look at the output ...
>>>
```

In [1]:
from Stock import Stock
goog = Stock('GOOG',100,490.10)
ibm  = Stock('IBM',50, 91.23)

In [2]:
goog.__dict__

{'name': 'GOOG', 'shares': 100, 'price': 490.1}

In [3]:
ibm.__dict__

{'name': 'IBM', 'shares': 50, 'price': 91.23}

#### **练习 5.2：修改实例属性**
尝试给上述其中一个实例添加新属性：

In [5]:
goog.date = '2025-10-26'
goog.__dict__
ibm.__dict__

{'name': 'IBM', 'shares': 50, 'price': 91.23}

在上述输出中，你会发现 `goog` 实例具有 `date` 属性，但是 `ibm` 实例没有。重要的是要注意，Python 对实例属性确实没有任何限制。例如，实例属性不限于 `__init__()` 方法中设置的属性。

尝试直接添加一个新的值到 `__dict__` 对象中：

In [6]:
goog.__dict__['time'] = '09:45:00'
goog.time

'09:45:00'

在这里，你会发现一个事实，实例仅仅是字典顶部的一层。注意：应该强调的是，直接操作字典并不常见——你应该始终使用语法 (.) 编写代码。
#### **练习 5.3：类的作用**
类中的定义被类的所有实例所共享。所有的实例都有一个链接，指向它们的关联类：

In [8]:
goog.__class__
ibm.__class__

Stock.Stock

尝试在实例上调用方法：

In [10]:
goog.cost()
ibm.cost()

4561.5

名字 'cost' 既不在 `goog.__dict__` 中定义，也不在 `ibm.__dict__`中定义。相反，而是由类字典提供的。请尝试以下代码：

In [12]:
Stock.__dict__['cost']

<function Stock.Stock.cost(self)>

尝试直接通过字典调用 `cost()` 方法：

In [14]:
Stock.__dict__['cost'](goog)
Stock.__dict__['cost'](ibm)

4561.5

你是如何调用类中定义的函数，那么 self 就是怎么调用实例的。

尝试给 Stock 类添加新属性：：

In [15]:
Stock.foo = 42

该新属性会出现在所有实例中：

In [17]:
goog.foo
ibm.foo

42

但是，foo 并不属于实例字典：

In [18]:
goog.__dict__

{'name': 'GOOG',
 'shares': 100,
 'price': 490.1,
 'date': '2025-10-26',
 'time': '09:45:00'}

你可以访问 `foo` 属性的原因是：当 Python 在实例字典中查找不到某个属性时，那么它就会到类字典中查找。

注意：本部分主要阐明什么是类变量。假设你有这样一个类：

In [19]:
class Foo(object):
    a = 13  # 类变量
    def __init__(self,b):
        self.b = b # 实例变量

在 Foo 类中，因为变量 `a` 在类体（body of the class）中被赋值，所以 `a` 是“类变量（class variable）”。变量 `a` 可以被 Foo 类的所有实例所共享。示例：

In [24]:
f = Foo(10)
g = Foo(20)
print(f.a,f.b)
print(g.a,g.b)
Foo.a = 42
print(f.a,g.a)

42 10
42 20
42 42


#### **练习 5.4：绑定方法**
Python 有一个微妙的特性：调用方法实际上涉及两个步骤以及一个称为绑定方法的东西。示例：

In [26]:
s = goog.sell
s
s(25)
goog.shares

实际上，绑定方法包含调用一个方法的所需的所有内容。例如，它们记录了实现方法的函数：

In [28]:
s.__func__

<function Stock.Stock.sell(self, nshares)>

这与在 `Stock `字典中找到的值是一样的：

In [29]:
Stock.__dict__['sell']

<function Stock.Stock.sell(self, nshares)>

绑定方法还记录实例，即`self`:

In [30]:
s.__self__

Stock(GOOG,75,490.1)

你可以使用 `()` 一起调用所有的函数。例如，调用 `s(25)` 实际是这样做的：

In [31]:
s.__func__(s.__self__,25)  # 和s(25)一样
goog.shares

50

#### **练习 5.5：继承**
创建一个继承自 `Stock` 的类：

In [33]:
class NewStock(Stock):
    def yow(self):
        print('Yow!')
n = NewStock('ACME',50,123.45)
n.cost()
n.yow()

Yow!


通过扩展属性的搜索过程来实现继承。`__bases__` 属性是一个包含直接父类的元组：

In [34]:
NewStock.__base__

Stock.Stock

`__mro__` 属性是一个包含所有父类的元组，父类按查找顺序排列。

In [35]:
NewStock.__mro__

(__main__.NewStock, Stock.Stock, object)

实例 `n` 是这找到 `cost()` 方法的：

In [36]:
for cls in n.__class__.__mro__:
    if 'cost' in cls.__dict__:
        break

In [38]:
cls
cls.__dict__['cost']

<function Stock.Stock.cost(self)>

#### **02 类和封装**
创建类时，通常会尝试将类的内部细节进行封装。本节介绍 Python 编程中有关封装的习惯用法（包括私有变量和私有属性）。
#### **Public vs Private**
虽然类的主要作用之一是封装对象的属性和内部实现细节。但是，类还定义了外界用来操作该对象的公有接口（public interface）。实现细节与公有接口之间的区别很重要。
#### **问题**
在 Python 中，几乎所有与类和对象有关的东西都是 **开放（open）** 的。
*  可以轻松地查看对象的内部细节。
*  可以随意地修改。
*  没有访问控制的概念（例如：私有类成员）。
如何隔离内部实现的细节，这是一个问题。
#### **Python 封装**
Python 依赖编程约定来指示某些东西的用途。这就约定基于命名。有一种普遍的态度是，程序员应该遵守规则，而不是让语言来强制执行规则。
#### **私有属性**
以下划线`_`开头的任何属性被认为是私有的（private）。
```
class Person(object):
    def __init__(self, name):
        self._name = 0
```
如前所述，这这是一种编程风格。你仍然可以对这些私有属性进行访问和修改。
```
>>> p = Person('Guido')
>>> p._name
'Guido'
>>> p._name = 'Dave'
>>>
```
一般来说，一个以下划线 `_` 开头的名称被认为是内部实现，无论该名称是变量名、函数名还是模块名。如果你发现自己直接使用这些名称，那么你可能在做一些错误的事情。你应该寻找更高级的功能。
#### **简单属性**
考虑下面这个类：
```
class Stock:
    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price
```
这里有一个让人惊讶的特性，你可以给属性设置任何值：
```
>>> s = Stock('IBM', 50, 91.1)
>>> s.shares = 100
>>> s.shares = "hundred"
>>> s.shares = [1, 0, 0]
>>>
```
你可能会想要对此进行检查（译注：例如 `shares` 表示的是股份数目，值应该是整数。所以给 `shares `赋值时应该对值进行检查。如果检查发现给 `shares` 赋的值不是整数，那么应该触发一个 `TypeError` 异常）：
```
s.shares = '50'     # Raise a TypeError, this is a string
```
这时候你会怎么做？
#### **托管属性**
方法一：引进访问方法（accessor methods）。
```
class Stock:
    def __init__(self, name, shares, price):
        self.name = name
	    self.set_shares(shares)
	    self.price = price

    # Function that layers the "get" operation
    def get_shares(self):
        return self._shares

    # Function that layers the "set" operation
    def set_shares(self, value):
        if not isinstance(value, int):
            raise TypeError('Expected an int')
        self._shares = value
```
糟糕的是，这破坏了我们的已有代码。例如：之前是通过 `s.shares = 50` 给 `shares` 赋值的，那么现在就要改成`s.set_shares(50)` 给 `shares` 赋值，这很不好。
#### **特征属性（Properties）**
方法二：
```
class Stock:
    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price

    @property
    def shares(self):
        return self._shares

    @shares.setter
    def shares(self, value):
        if not isinstance(value, int):
            raise TypeError('Expected int')
        self._shares = value
```
现在，普通属性（normal attribute）的访问触发了 `@property` 和 `@shares.setter` 下的 getter 方法和 setter 方法。
```
>>> s = Stock('IBM', 50, 91.1)
>>> s.shares         # Triggers @property
50
>>> s.shares = 75    # Triggers @shares.setter
>>>
```
使用该方法，不需要对源代码做任何修改。在类内（包括在 `__init__()` 方法内）有赋值的时候，直接调用新的 setter：
```
class Stock:
    def __init__(self, name, shares, price):
        ...
        # This assignment calls the setter below
        self.shares = shares
        ...

    ...
    @shares.setter
    def shares(self, value):
        if not isinstance(value, int):
            raise TypeError('Expected int')
        self._shares = value
```
特征属性和私有名称（ private names）的使用之间经常会出现混淆。尽管特征属性内部使用的是私有名称，如 `_shares`。类的其它地方（不是特征属性），仍可以继续使用诸如 `shares` 这样的名称。

特征属性对于计算数据属性也非常有用。
```
class Stock:
    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price

    @property
    def cost(self):
        return self.shares * self.price
    ...
```
这允许你删除 cost 后面的括号，隐藏 cost 是一个方法的事实：
```
>>> s = Stock('GOOG', 100, 490.1)
>>> s.shares # Instance variable
100
>>> s.cost   # Computed Value
49010.0
>>>
```

#### **统一访问**
最后一个例子展示了如何在对象上放置一个更加统一的接口。如果不这样做，对象使用起来可能会令人困惑。
```
>>> s = Stock('GOOG', 100, 490.1)
>>> a = s.cost() # Method
49010.0
>>> b = s.shares # Data attribute
100
>>>
```
为什么 cost 后面需要加上括号 `()`，但是 shares 却不需要？ 特征属性可以解决这个问题。
#### **装饰器语法**
`@` 语法称为“装饰（decoration）”。它指定了一个修饰符（modifier），应用于紧接其后的函数定义：
```
...
@property
def cost(self):
    return self.shares * self.price
```
更多细节在 第 7 节 中给到。
#### **插槽属性（`__slots__`）**
你可以使用 `__slots__` 限制属性名称集：
```
class Stock:
    __slots__ = ('name','_shares','price')
    def __init__(self, name, shares, price):
        self.name = name
        ...
```
使用其它属性时，将会触发错误：
```
>>> s.price = 385.15
>>> s.prices = 410.2
Traceback (most recent call last):
File "<stdin>", line 1, in ?
AttributeError: 'Stock' object has no attribute 'prices'
```
尽管这样可以防止错误和限制对象的使用，但实际上使用 `__slots__` 是为了提高性能，提高 Python 利用内存的效率。
#### **关于封装的最终说明**
不要滥用私有属性（private attributes），特征属性（properties），插槽属性（slots）等。它们有特殊的用途，你在阅读其它 Python 代码时可能会看到。但是，对于大多数日常编码而言，它们不是必需的。

### 练习
#### **练习 5.6：简单特征属性**
使用特征属性是一种非常有用的给对象添加“计算属性”的方式。虽然你在 `stock.py` 文件中创建了 `Stock` 对象，但是请注意，在 `Stock` 对象上 ，对于不同类型的属性，获取方式稍微有点不同。

In [41]:
from Stock import Stock
s = Stock('GOOG', 100, 490.1)
s.shares
s.price
s.cost()

49010.0

具体来说，`cost` 后面之所以要添加括号，是因为 `cost` 是一个方法。

如果你想去掉 `cost()` 的括号，那么可以把该方法转为一个特征属性。请修改 `Stock` 类，使其像下面这样计算所持有股票的总价:

In [1]:
from Stock import Stock
s = Stock('GOOG', 100, 490.1)
s.cost

49010.0

尝试将 `cost`作为方法调用（`s.cost()`），你会发现，现在已经被定义为特征属性的 `cost` 无法作为方法被调用。

In [2]:
s.cost()

TypeError: 'float' object is not callable

这些更改很可能会破坏你之前的 `pcost.py` 程序，所以，你可能需要返回到 `pcost.py` 中去掉 cost() 方法后面的括号`()`。
#### **练习 5.7：特征属性和 Setters**
请修改 `shares` 属性，以便将该值存储在私有属性中，并且使用属性函数（property functions）确保赋给 `shares` 的值总是整数。预期行为示例：

In [ ]:
from Stock import Stock
s = Stock('GOOG', 100, 490.1)
s.shares = 50
s.shares = 'a lot'

#### **练习 5.8：添加插槽属性（slots）**
请修改 `Stock` 类，以便 `Stock` 类拥有一个 `__slots__` 属性。然后确认无法添加新属性:

In [2]:
from Stock import Stock
s = Stock('GOOG', 100, 490.1)
s.name
s.blah = 42

AttributeError: 'Stock' object has no attribute 'blah' and no __dict__ for setting new attributes

使用 `__slots__` 时，Python 使用更高效的对象内部表示。如果你尝试查看实例 `s` 的底层字典会发生什么？

In [3]:
s.__dict__

AttributeError: 'Stock' object has no attribute '__dict__'

应当指出， `__slots__` 作为数据结构是类中最常用的一种优化。使用插槽属性使程序占用更少的内存，运行更快。但是，在其它大多数类中，你应该尽可能避免使用 `__slots__` 。